In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter

import numpy as np
import time
from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io

from smt.sampling_methods import LHS
from scipy.io import savemat

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

    

cuda:0


In [2]:
def true_1D_1(x): #True function for 1D_1 dy2/dx2 + dy/dx - 6y = 0; BC1: y(0)=2; BC2: dy/dx at (x=0) = -1;
    y = np.exp(2*x) + np.exp(-3*x)
    return y
    

In [3]:
label = "1D_SODE_atanh"
loss_thresh = 0.005

x = np.linspace(0,2,100).reshape(-1,1)

bc1_x = x[0].reshape(-1,1)
bc1_y = true_1D_1(x[0]).reshape(-1,1)
x_bc1_train = torch.from_numpy(bc1_x).float().to(device)
y_bc1_train = torch.from_numpy(bc1_y).float().to(device)
    

bc2_x = x[0].reshape(-1,1)
x_bc2_train = torch.from_numpy(bc2_x).float().to(device)
bc2_val = torch.tensor(-1.0,device=device)
bc2_val = bc2_val.view(1,1)

x_test = x.reshape(-1,1)
x_test_tensor = torch.from_numpy(x_test).float().to(device)
y_true = true_1D_1(x_test)
y_true_norm = np.linalg.norm(y_true,2)

# Domain bounds
lb = np.array(x[0]) 
ub = np.array(x[-1]) 

In [4]:
def colloc_pts(N_f,seed):
    #Collocation Points
    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,y)
    x01 = np.array([[0.0, 1.0]])
    sampling = LHS(xlimits=x01,random_state =seed)
    
    x_coll_train = lb + (ub-lb)*sampling(N_f)
    x_coll_train = np.vstack((x_coll_train, bc1_x.reshape(-1,1))) # append training points to collocation points 

    return x_coll_train

In [5]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers):
        super().__init__() #call __init__ from parent class 
              
    
        self.activation = nn.Tanh()
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data) 
        
        self.alpha = Parameter(torch.ones((50,len(layers)-2)))
        self.alpha.requiresGrad = True
        
        self.n = torch.tensor(3.0)
    
    'forward pass'
    def forward(self,x):
        if torch.is_tensor(x) != True:         
            x = torch.from_numpy(x)                
        
        u_b = torch.from_numpy(ub).float().to(device)
        l_b = torch.from_numpy(lb).float().to(device)
                      
        #preprocessing input 
        x = (x - l_b)/(u_b - l_b) #feature scaling
        
        #convert to float
        a = x.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a = self.activation(self.n*self.alpha[:,i]*z)
            
        a = self.linears[-1](a) 
         
        return a
                        
    def loss_BC1(self,x,y):
                
        loss_bc1 = self.loss_function(self.forward(x), y)
                
        return loss_bc1
    
    def loss_BC2(self,x_bc2,bc2_val):
        g = x_bc2.clone()             
        g.requires_grad = True
        y = self.forward(g)    
            
        y_x = autograd.grad(y,g,torch.ones([x_bc2.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        dy_dx = y_x[:,[0]]
        
        bc2 = dy_dx
        
        loss_bc2= self.loss_function(bc2,bc2_val)

        return loss_bc2
    
    def loss_PDE(self, x_coll,f_hat):
             
        g = x_coll.clone()             
        g.requires_grad = True
  
        y = self.forward(g) 

        y_x = autograd.grad(y,g,torch.ones([x_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        y_xx = autograd.grad(y_x,g,torch.ones(x_coll.shape).to(device), create_graph=True,allow_unused = True)[0]

        dy_dx = y_x[:,[0]]
        
        dy2_d2x = y_xx[:,[0]]
        
        f = dy2_d2x + dy_dx - 6*y
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    
    def loss(self,x_bc1,y_bc1,x_bc2,bc2_val,x_coll,f_hat):

        loss_bc1 = self.loss_BC1(x_bc1,y_bc1)
        loss_bc2 = self.loss_BC2(x_bc2,bc2_val)
        loss_f = self.loss_PDE(x_coll,f_hat)
        
        loss_val = loss_bc1 + loss_bc2 + loss_f
        
        return loss_val
          
    'test neural network'
    
    def test(self):
        y_pred = self.forward(x_test_tensor)
        y_pred = y_pred.cpu().detach().numpy()

        return y_pred
    
    def test_loss(self):
        y_pred = self.test()
        
        test_mse = np.mean(np.square(y_pred.reshape(-1,1) - y_true.reshape(-1,1)))
        test_re = np.linalg.norm(y_pred.reshape(-1,1) - y_true.reshape(-1,1),2)/y_true_norm
        
        return test_mse, test_re

In [6]:
def train_step(x_coll,f_hat):
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(x_bc1_train,y_bc1_train,x_bc2_train,bc2_val,x_coll,f_hat)
        loss.backward()
        
        return loss

    optimizer.step(closure)

In [7]:
def data_update(loss_np):
    train_loss.append(loss_np)
    alpha_val.append(PINN.alpha.cpu().detach().numpy())
    
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [8]:
def train_model(max_iter,rep):
    print(rep) 
    torch.manual_seed(rep*123)
    start_time = time.time()
    thresh_flag = 0

    for i in range(max_iter):
        x_coll = torch.from_numpy(colloc_pts(N_f,i*11)).float().to(device)
        f_hat = torch.zeros(x_coll.shape[0],1).to(device)
        train_step(x_coll,f_hat)
        
        loss_np = PINN.loss(x_bc1_train,y_bc1_train,x_bc2_train,bc2_val,x_coll,f_hat).cpu().detach().numpy()
        if(thresh_flag == 0):
            if(loss_np < loss_thresh):
                time_threshold[rep] = time.time() - start_time
                epoch_threshold[rep] = i+1            
                thresh_flag = 1       
        data_update(loss_np)
        print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1])
    
    elapsed_time[rep] = time.time() - start_time
    print('Training time: %.2f' % (elapsed_time[rep]))

In [9]:
max_reps = 10
max_iter = 100

N_f = 1000

train_loss_full = []
test_mse_full = []
test_re_full = []
alpha_full = []
elapsed_time= np.zeros((max_reps,1))

time_threshold = np.empty((max_reps,1))
time_threshold[:] = np.nan
epoch_threshold = max_iter*np.ones((max_reps,1))

for reps in range(max_reps):
    
    train_loss = []
    test_mse_loss = []
    test_re_loss =[]
    alpha_val = []
    
    'Generate Training data'
    torch.manual_seed(reps*36)
     #Total number of collocation points 
    
    
    layers = np.array([1,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers
    PINN = Sequentialmodel(layers)
    PINN.to(device)

    'Neural Network Summary'
    print(PINN)

    params = list(PINN.parameters())
    
    optimizer = torch.optim.LBFGS(PINN.parameters(), lr=0.5, 
                              max_iter = 10, 
                              max_eval = 15, 
                              tolerance_grad = 1e-5, 
                              tolerance_change = 1e-5, 
                              history_size = 100, 
                              line_search_fn = 'strong_wolfe')
   

    start_time = time.time()
    train_model(max_iter,reps)

    
    torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
    train_loss_full.append(train_loss)
    test_mse_full.append(test_mse_loss)
    test_re_full.append(test_re_loss)
    alpha_full.append(alpha_val)    
    
    print('Training time: %.2f' % (elapsed_time[reps]))

mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full, "test_re_loss": test_re_full, "Time": elapsed_time, "alpha": alpha_full, "label": label,"Thresh Time": time_threshold,"Thresh epoch": epoch_threshold}
savemat(label+'.mat', mdic)

Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 26800.29 Test MSE 395.81081640344325 Test RE 1.014159846467565
1 Train Loss 4031.3628 Test MSE 395.0983107052206 Test RE 1.0132466322231892
2 Train Loss 1835.0089 Test MSE 396.88273673120943 Test RE 1.0155321732141342
3 Train Loss 970.9157 Test MSE 398.6388093802865 Test RE 1.0177763876

97 Train Loss 2.3114698 Test MSE 372.2683000151958 Test RE 0.9835367919405313
98 Train Loss 2.3114629 Test MSE 372.26531333473247 Test RE 0.9835328465119073
99 Train Loss 2.3109725 Test MSE 372.19219370334497 Test RE 0.9834362499605448
Training time: 84.55
Training time: 84.55
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
1
0 Train Loss 16571.096

93 Train Loss 2.3620045 Test MSE 379.56990849387296 Test RE 0.9931354175038954
94 Train Loss 2.3619478 Test MSE 379.557418956586 Test RE 0.9931190780844803
95 Train Loss 2.3619423 Test MSE 379.5539482196906 Test RE 0.9931145374501341
96 Train Loss 2.3619263 Test MSE 379.5507188744391 Test RE 0.9931103126007386
97 Train Loss 2.361916 Test MSE 379.5505531422906 Test RE 0.9931100957781698
98 Train Loss 2.3619072 Test MSE 379.5497893354568 Test RE 0.9931090965112037
99 Train Loss 2.3619008 Test MSE 379.55153234679057 Test RE 0.9931113768423645
Training time: 86.35
Training time: 86.35
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_

89 Train Loss 2.1373932 Test MSE 343.9549212310219 Test RE 0.94539509852561
90 Train Loss 2.1333184 Test MSE 343.45740765153374 Test RE 0.9447111177524227
91 Train Loss 2.1314952 Test MSE 342.9177301923662 Test RE 0.9439686097725525
92 Train Loss 2.1292102 Test MSE 342.45457578855553 Test RE 0.9433309190019827
93 Train Loss 2.1270747 Test MSE 341.5715308992499 Test RE 0.9421139090831311
94 Train Loss 2.1215765 Test MSE 339.56819021272474 Test RE 0.9393470645187629
95 Train Loss 2.108943 Test MSE 337.9657482746063 Test RE 0.9371280269056257
96 Train Loss 2.0988476 Test MSE 336.9748925335501 Test RE 0.935753271797679
97 Train Loss 2.096106 Test MSE 336.84408682836624 Test RE 0.9355716354948072
98 Train Loss 2.0960383 Test MSE 336.8480703817919 Test RE 0.9355771675624913
99 Train Loss 2.0947406 Test MSE 336.69741105891313 Test RE 0.9353679201312877
Training time: 116.61
Training time: 116.61
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    

85 Train Loss 3.1476762 Test MSE 384.58102779103274 Test RE 0.9996696575717601
86 Train Loss 3.1086066 Test MSE 384.534998591796 Test RE 0.9996098322457415
87 Train Loss 3.0827692 Test MSE 384.33738616626675 Test RE 0.9993529496354252
88 Train Loss 3.0788765 Test MSE 384.3032148156494 Test RE 0.9993085225248373
89 Train Loss 3.046501 Test MSE 384.3275283396746 Test RE 0.999340133407279
90 Train Loss 3.0152085 Test MSE 384.25283507424433 Test RE 0.999243018837786
91 Train Loss 2.9383929 Test MSE 384.60607870563166 Test RE 0.9997022153809273
92 Train Loss 2.8489468 Test MSE 385.2471616307638 Test RE 1.0005350482580584
93 Train Loss 2.7853136 Test MSE 384.458786720112 Test RE 0.9995107698566218
94 Train Loss 2.7386038 Test MSE 383.7158349253998 Test RE 0.9985445448035699
95 Train Loss 2.73252 Test MSE 383.61301492407176 Test RE 0.9984107514845917
96 Train Loss 2.7316573 Test MSE 383.6042646418583 Test RE 0.9983993644547977
97 Train Loss 2.7309546 Test MSE 383.6066020319195 Test RE 0.99840

...

78 Train Loss 2.3702657 Test MSE 381.29171959780894 Test RE 0.9953854071823038
79 Train Loss 2.367306 Test MSE 381.0791389536222 Test RE 0.9951078910947748
80 Train Loss 2.364019 Test MSE 380.6928167324832 Test RE 0.9946033636472609
81 Train Loss 2.362665 Test MSE 380.5187821246051 Test RE 0.9943759950366082
82 Train Loss 2.3616104 Test MSE 380.43810094878455 Test RE 0.9942705709664299
83 Train Loss 2.3609283 Test MSE 380.1996794201779 Test RE 0.9939589661859769
84 Train Loss 2.3605313 Test MSE 380.2029972690285 Test RE 0.9939633031154872
85 Train Loss 2.3592849 Test MSE 379.9209380887363 Test RE 0.993594541569063
86 Train Loss 2.358741 Test MSE 379.81744427959086 Test RE 0.9934592004000109
87 Train Loss 2.3572807 Test MSE 379.77485992922647 Test RE 0.9934035065381616
88 Train Loss 2.3564093 Test MSE 379.584813421575 Test RE 0.9931549165025658
89 Train Loss 2.3535357 Test MSE 378.8905782054395 Test RE 0.9922462939273713
90 Train Loss 2.3526354 Test MSE 378.6985154318796 Test RE 0.99199

74 Train Loss 2.3039927 Test MSE 370.3160245967111 Test RE 0.980954435679098
75 Train Loss 2.3032212 Test MSE 370.27216109787946 Test RE 0.9808963375073778
76 Train Loss 2.3022826 Test MSE 370.1621461140291 Test RE 0.9807506050934423
77 Train Loss 2.3012762 Test MSE 369.9840624304949 Test RE 0.9805146589090099
78 Train Loss 2.3010697 Test MSE 369.9624007955917 Test RE 0.9804859551571393
79 Train Loss 2.3004963 Test MSE 369.92113729476915 Test RE 0.9804312747205399
80 Train Loss 2.2997842 Test MSE 369.8372158210466 Test RE 0.9803200565496603
81 Train Loss 2.2993944 Test MSE 369.69408663048483 Test RE 0.9801303433620145
82 Train Loss 2.2992065 Test MSE 369.67704123225724 Test RE 0.9801077477820483
83 Train Loss 2.2986922 Test MSE 369.57058787646963 Test RE 0.9799666201768128
84 Train Loss 2.2982295 Test MSE 369.42956327547006 Test RE 0.9797796293927169
85 Train Loss 2.2981858 Test MSE 369.4156874057247 Test RE 0.9797612288320535
86 Train Loss 2.2981791 Test MSE 369.4061906311874 Test RE 

70 Train Loss 2.3849049 Test MSE 383.0803181510032 Test RE 0.9977172988005293
71 Train Loss 2.3848722 Test MSE 383.0776973039245 Test RE 0.9977138858492282
72 Train Loss 2.3839784 Test MSE 383.02824396318283 Test RE 0.997649483930762
73 Train Loss 2.3839784 Test MSE 383.02824396318283 Test RE 0.997649483930762
74 Train Loss 2.3839784 Test MSE 383.02824396318283 Test RE 0.997649483930762
75 Train Loss 2.3839784 Test MSE 383.02824396318283 Test RE 0.997649483930762
76 Train Loss 2.3839767 Test MSE 383.0268331179008 Test RE 0.9976476465592551
77 Train Loss 2.3839319 Test MSE 383.02539377714135 Test RE 0.9976457720740415
78 Train Loss 2.3839312 Test MSE 383.02292683981307 Test RE 0.9976425593189625
79 Train Loss 2.3838937 Test MSE 383.02055211301786 Test RE 0.9976394666422274
80 Train Loss 2.3838928 Test MSE 383.02055171531424 Test RE 0.9976394661242853
81 Train Loss 2.3838758 Test MSE 383.0177139263145 Test RE 0.9976357703754101
82 Train Loss 2.3838613 Test MSE 383.01492923006634 Test RE 

66 Train Loss 77.597565 Test MSE 428.0347546112539 Test RE 1.054634803529735
67 Train Loss 74.86167 Test MSE 426.90010884749165 Test RE 1.0532360491398183
68 Train Loss 72.51281 Test MSE 425.6123999762089 Test RE 1.0516463497184025
69 Train Loss 69.91926 Test MSE 424.006930901882 Test RE 1.0496609976608418
70 Train Loss 67.353134 Test MSE 423.3113490567537 Test RE 1.048799661715529
71 Train Loss 65.10994 Test MSE 422.96612103470676 Test RE 1.0483719045983662
72 Train Loss 63.52965 Test MSE 423.1790903799303 Test RE 1.048635806325973
73 Train Loss 62.194435 Test MSE 423.8564383961034 Test RE 1.0494747033795069
74 Train Loss 60.905678 Test MSE 423.0025640473013 Test RE 1.0484170677990334
75 Train Loss 60.677925 Test MSE 422.90682992940276 Test RE 1.0482984219901728
76 Train Loss 60.60974 Test MSE 422.81487125947865 Test RE 1.0481844425475026
77 Train Loss 60.581985 Test MSE 422.7898843549317 Test RE 1.0481534700423318
78 Train Loss 60.079468 Test MSE 422.95609588886344 Test RE 1.04835948

63 Train Loss 2.2896721 Test MSE 367.4664222156477 Test RE 0.9771728974049864
64 Train Loss 2.2850595 Test MSE 367.1029780719762 Test RE 0.9766895394653544
65 Train Loss 2.281189 Test MSE 366.8684142831053 Test RE 0.9763774572703658
66 Train Loss 2.2780566 Test MSE 366.479361552769 Test RE 0.9758596107314944
67 Train Loss 2.2756093 Test MSE 365.7842232466457 Test RE 0.974933665848025
68 Train Loss 2.2727418 Test MSE 365.46684824107865 Test RE 0.9745106203685372
69 Train Loss 2.2687185 Test MSE 364.9033375759926 Test RE 0.9737590351354302
70 Train Loss 2.2639506 Test MSE 364.1933545076694 Test RE 0.9728112649034675
71 Train Loss 2.2618182 Test MSE 364.07792303660995 Test RE 0.9726570859031635
72 Train Loss 2.2591245 Test MSE 363.4112057161562 Test RE 0.9717660891651492
73 Train Loss 2.2562559 Test MSE 362.6705708137574 Test RE 0.9707753506488903
74 Train Loss 2.2528772 Test MSE 361.2490678692347 Test RE 0.9688709848272566
75 Train Loss 2.2494118 Test MSE 360.89852131730294 Test RE 0.968

In [10]:
a = 0
for i in range(10):
    a = a + test_re_full[i][-1]
print(a/10)

0.9861951731359333
